# 仕組み
1、評価と選定
３要素（駒の価値、成るまでの距離、王（玉）までの距離　）でどの駒

まずコンピュータが出せるすべての手を出して（それぞれに対する点数もつけて）、コンピュータのすべての手に対して、プレイヤーのすべての手も出して（点数をつけて）、コンピュータの手の点　ー　プレーヤーの手の点を計算して、この式で一番高い点が出せるコンピュータの点を導出する。

# 各オブジェクト説明

kind = {0:"ou", 1:"gyoku", 2:"hisha", 3:"kaku",4:"kin", 5:"gin", 6:"keima", 7:"kyousha", 8:"hu"}　#駒の種類を特定 （自分は王、相手は玉）

way = {"ou":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]], 
       "gyoku":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]], 
       "hisha":[[-1, 0], [0, 1], [1, 0], [0, -1]], 
       "kaku":[[-1, 1], [1, 1], [1, -1], [-1, -1]], 
       "kin":[[-1, 1], [0, 1], [1, 1], [1, 0], [0, -1], [-1, 0]], 
       "gin":[[-1, 1], [0, 1], [1, 1], [1, -1], [-1, -1]], 
       "keima":[[-1, 2], [1, 2]], 
       "kyousha":[[0, 1]], 
       "hu":[[0, 1]], 
       "ryu":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]], 
       "uma":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]],
      } #各駒の進む方向

value_k = {"ou":[8, 8], "gyoku":[8, 8], "hisha":[12, 16], "kaku":[12, 16],"kin":[6, 6], "gin":[5, 6], "keima":[6, 6], "kyousha":[3, 6], "hu":[1, 6]} #各駒の評価係数
value_p = {"first":1, "second":2, "third":3} #各列の評価係数（相手陣地に近いほど高い）
value_d = {"first":2, "second":1} #王との距離の評価係数
end_f = 0 #対局終了フラグ
player = [0, 2, 4] #各プレーヤの持ち駒
computer = [3, 1, 6]

stage = {0:[1, 0], 2:[0, 1], 4:[2, 0], 3:[0, 2], 1:[1, 2], 6:[2, 2]} #初期配置


# memo 
駒にプレイヤの識別タグ？を付けなければいけない
将来DBに駒情報とかのパラメータを置くということを考えながら設計する。

In [1]:
import numpy as np

In [2]:
kind = {0:"ou", 1:"gyoku", 2:"hisha", 3:"kaku",4:"kin", 5:"gin", 6:"keima", 7:"kyousha", 8:"hu"}

way = {"ou":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]], 
       "gyoku":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]], 
       "hisha":[[-1, 0], [0, 1], [1, 0], [0, -1]], 
       "kaku":[[-1, 1], [1, 1], [1, -1], [-1, -1]], 
       "kin":[[-1, 1], [0, 1], [1, 1], [1, 0], [0, -1], [-1, 0]], 
       "gin":[[-1, 1], [0, 1], [1, 1], [1, -1], [-1, -1]], 
       "keima":[[-1, 2], [1, 2]], 
       "kyousha":[[0, 1]], 
       "hu":[[0, 1]], 
       "ryu":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]], 
       "uma":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]],
      } 

value_k = {"ou":[8, 8], "gyoku":[8, 8], "hisha":[12, 16], "kaku":[12, 16],"kin":[6, 6], "gin":[5, 6], "keima":[6, 6], "kyousha":[3, 6], "hu":[1, 6]}
value_l = {"first":1, "second":2, "third":3} 
value_d = {"first":2, "second":1}
end_f = 0
#player = [0, 2, 5] #各プレーヤの持ち駒
#computer = [3, 1, 6]


#stage = {0:[1, 0], 2:[0, 1], 5:[2, 0], 3:[0, 2], 1:[1, 2], 6:[2, 2]} #初期盤面（自分が下）後で自分とcpの駒の区別をつけられるようにしなきゃだめ
points =[]
points2 =[]

# 1, cp,pl共に使う共通の評価関数。

In [3]:
def rob(player, nextpos, stage): #相手プレーヤーの手持ちコマがplayerに入る
    enemyplaces = {}
    for i in player:
        enemyplaces[i] = stage[i]
    for i in enemyplaces.keys():
        if enemyplaces[i] == nextpos:
            return i #なんか0の駒（王）の時pythonだと0 = Falseみたいな感じで処理されるから数値そのままだとダメくさい。
    return "false"  #上のやつを変えるのめんどいからFalseの時文字列のfalseにした 
    

In [4]:
#rob(player, )

In [5]:
def evaluation_r(k, l, d, r) : #点数の合計をだすよ～
    return value_k[kind[k]][0] + value_l[l] + value_d[d] + value_k[kind[r]][0]#まだ成ることは考えてないよ～（value_k[kind[0]][0]の０のとこ）

In [6]:
def evaluation_n(k, l, d) : #点数の合計をだすよ～
    return value_k[kind[k]][0] + value_l[l] + value_d[d]#まだ成ることは考えてないよ～（value_k[kind[0]][0]の０のとこ）

In [7]:
def lin(me): #列の点数を出すよ～
    if me[1] == 2:
        return "third"
    elif me[1] == 1:
        return "second"
    elif me[1] == 0:
        return "first"

In [8]:
def dist(me, king): #王までの距離の点数を出すよ～
    dist = np.sqrt((king[0] - me[0]) ** 2 + (king[1] - me[1]) ** 2)
    if dist < 2 :
        return "first"
    elif dist >= 2 and dist <= 4 :
        return "second"

In [9]:
def movedstage(tomovestage, choices):
    stageset = []
    originstage = {}
    for i in range(len(choices)):
        for k, v in tomovestage.items():
            originstage[k] = v
        originstage[choices[i][1]] = choices[i][2]
        stageset.append([choices[i][0], originstage])
        #print (originstage)
    return stageset

originstageにtomovestage(globalのstage)を直接代入するとstageが変更されるからfor文やってる

In [10]:
#print("stage=", stage)
#print (movedstage(stage, [[16, 3, [1, 1]], [12, 1, [2, 1]], [12, 1, [1, 1]], [24, 1, [0, 1], 'robable_piece=', 2], [18, 6, [1, 0], 'robable_piece=', 0]]))
#print("stage=", stage)

# 2, コンピュータの手をシミュレーションする関数

In [11]:
def cp_simu(computer):
    for piece in computer : #pieceを選択
        #print("*piece=", piece)
        for w in way[kind[piece]]: #wayを選択
            #print("way=", w)
            reverse_w = list(map(lambda x : x * -1, w))
            position = np.array(stage[piece]) #選んだ駒の現在の座標
            next_position = position + np.array(reverse_w) #進んだ先の座標
            l_next_position = next_position.tolist() #numpyの配列と標準のリストの形が違うからinとかやってもダメ臭い=>ndarray.tolist()使おうや
            #print(list(map(lambda x:x >= 0 and x <= 2, l_next_position)))
            if all(map(lambda x:x >= 0 and x <= 2, l_next_position)) == True: #進んだ先が盤面内に収まっているか
                if l_next_position not in stage.values(): #進んだ先に駒がなかった場合
                    king_position = stage[0] #自分は王だから王のポジションを特定する
                    line = str(lin(next_position)) #列の評価係数(function_lin)
                    #print("line=", line)
                    distance = str(dist(l_next_position, king_position)) #距離の評価係数(function_dist)
                    #print("l_next_position=", l_next_position)
                    #print("distance=", distance)
                    point = evaluation_n(piece, line, distance)
                    #print("*point=", point) #手の駒の能力×列×距離の点数(function_evaluation)
                    points.append([point, piece, l_next_position]) #動かした駒、動かした結果の評価点数、動かした場合その駒の位置
                else:
                    #print("もうおるやんけ！")#進んだ先が相手の駒だった場合は点数加算＆駒の位置の得点も加算
                    robable_piece = rob(player, l_next_position, stage) #取れる駒の種類(function_rob)
                    if robable_piece != "false":
                        #進んだ先の駒が敵駒だったばあい
                        #print("robable_piece=", robable_piece)
                        #print("line=", line)
                        #print("l_next_position=", l_next_position)
                        #print("distance=", distance)
                        point = evaluation_r(piece, line, distance, robable_piece)
                        #print("*駒取った+point=", point) #手の駒の能力+列+距離の点数+取った駒の能力(function_evaluation)
                        points.append([point, piece, l_next_position,"robable_piece=", robable_piece]) 
                        #動かした駒、動かした結果の評価点数、動かした場合その駒の位置、手に入れた駒
                    #else:
                        #print("味方やんけ")
            #else:
                #print("進めないよ")
    
    return points
            

In [12]:
#print (cp_simu(computer))


# 3, プレイヤーの手をシミュレートする関数

In [13]:
def pl_simu(prestage, player):
    stage = prestage
    
    for piece in player : #pieceを選択
        #print("*piece=", piece)
        for w in way[kind[piece]]: #wayを選択
            #print("way=", w)
            position = np.array(stage[piece]) #選んだ駒の現在の座標
            next_position = position + np.array(w) #進んだ先の座標
            l_next_position = next_position.tolist() #numpyの配列と標準のリストの形が違うからinとかやってもダメ臭い=>ndarray.tolist()使おうや
            #print(list(map(lambda x:x >= 0 and x <= 2, l_next_position)))
            if all(map(lambda x:x >= 0 and x <= 2, l_next_position)) == True: #進んだ先が盤面内に収まっているか
                if l_next_position not in stage.values(): #進んだ先に駒がなかった場合
                    king_position = stage[1] #コンピュータは玉だから玉のポジションを特定する
                    line = str(lin(next_position)) #列の評価係数(function_lin)
                    #print("line=", line)
                    distance = str(dist(l_next_position, king_position)) #距離の評価係数(function_dist)
                    #print("l_next_position=", l_next_position)
                    #print("distance=", distance)
                    point = evaluation_n(piece, line, distance)
                    #print("*point=", point) #手の駒の能力×列×距離の点数(function_evaluation)
                    points2.append([point, piece, l_next_position]) #動かした駒、動かした結果の評価点数、動かした場合その駒の位置
                else:
                    #print("もうおるやんけ！")#進んだ先が相手の駒だった場合は点数加算＆駒の位置の得点も加算
                    robable_piece = rob(computer, l_next_position, stage) #取れる駒の種類(function_rob)
                    if robable_piece != "false":
                        #進んだ先の駒が敵駒だったばあい
                        #print("robable_piece=", robable_piece)
                        #print("line=", line)
                        #print("l_next_position=", l_next_position)
                        #print("distance=", distance)
                        point = evaluation_r(piece, line, distance, robable_piece)
                        #print("*駒取った+point=", point) #手の駒の能力+列+距離の点数+取った駒の能力(function_evaluation)
                        points2.append([point, piece, l_next_position,"robable_piece=", robable_piece]) 
                        #動かした結果の評価点数、動かした駒、動かした場合その駒の位置、手に入れた駒
                    #else: 
                        #print("味方やんけ")
                     
            #else:
                #print("進めないよ")
    
    #print("points2=", points2)
    return points2

In [14]:
#print (pl_simu(stage, player))

# 4, コンピュータの手とその次のプレーヤーの手の評価をして、コンピュータはどの手を出せばいいのかを選ぶ関数。

In [15]:
"""
def selectway():
    cp_movedstage = movedstage(stage, cp_simu(computer))
    #print("cp_movedstage=", cp_movedstage)
    finaly_points = []
    pl_movedstage = []
    n = 0
    for i in cp_movedstage:
        plways = pl_simu(i[1], player)
        pl_movedstage.append((movedstage(stage, pl_simu(i[1], player)), "num= ¥n", n))
        finaly_points.append(i[0] - max(plways)[0])  #コンピュータの手の点　ー　プレイヤーの手の点
        n += 1
        #print("plways=", plways)
        #print("finaly_points=", finaly_points)
    #print("pl_movedstage=", pl_movedstage)
    return cp_movedstage[finaly_points.index(max(finaly_points))][1]
"""

'\ndef selectway():\n    cp_movedstage = movedstage(stage, cp_simu(computer))\n    #print("cp_movedstage=", cp_movedstage)\n    finaly_points = []\n    pl_movedstage = []\n    n = 0\n    for i in cp_movedstage:\n        plways = pl_simu(i[1], player)\n        pl_movedstage.append((movedstage(stage, pl_simu(i[1], player)), "num= ¥n", n))\n        finaly_points.append(i[0] - max(plways)[0])  #コンピュータの手の点\u3000ー\u3000プレイヤーの手の点\n        n += 1\n        #print("plways=", plways)\n        #print("finaly_points=", finaly_points)\n    #print("pl_movedstage=", pl_movedstage)\n    return cp_movedstage[finaly_points.index(max(finaly_points))][1]\n'

In [16]:
def selectway(stg, plr, cp):
    cpways = movedstage(stg,  cp_simu(stg, plr, cp))
    finaly_points = []
    for i in cpways:
        plways = pl_simu(i[1],  plr, cp)
        finaly_points.append(i[0] - max(plways)[0])
    return cpways[finaly_points.index(max(finaly_points))][1]

In [17]:
#pl_simu({0: [1, 0], 1: [1, 2], 2: [0, 1], 3: [0, 2], 5: [2, 0], 6: [1, 0]}, player)

In [18]:
#movedstage(stage, pl_simu({0: [1, 0], 1: [1, 2], 2: [0, 1], 3: [0, 2], 4: [2, 0], 6: [1, 0]}, player))

詰み判定でもselectwaysを使いたいので小変更をする

In [19]:
%time
print("nextstage", selectway({0:[1, 0], 2:[0, 1], 5:[2, 0], 3:[0, 2], 1:[1, 2], 6:[2, 2]}, [0, 2, 5], [3, 1, 6]))
#print("originstage=", stage)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


TypeError: cp_simu() takes 1 positional argument but 3 were given

In [ ]:
def decide_tumi():
    ways = selectway()

In [ ]:
def test():
    stein = []
    stein.append(1)
    return stein

In [ ]:
print(test())
print(test())

In [ ]:
player = [0, 2, 4] #各プレーヤの持ち駒
computer = [0, 1, 6]

stage = {player[0]:[1, 0],  #player[0] = 0
         player[1]:[0, 1],
         player[2]:[2, 0],
         computer[0]:[0, 2],
         computer[1]:[1, 2],
         computer[2]:[2, 2]} 


In [ ]:
stage[player[1]]